# Vault Transfers

*Work in Progress*

This notebook aims to provide a brief demonstration of how this codebase could be used for visualize vault transfers.

In [1]:
"""load dependencies and define constants"""
import os
import sys
import numpy as np
import pandas as pd
from decimal import Decimal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from typing import Literal

sys.path.append("..")
from src.yearn import Network, Yearn, Subgraph, Vault
from src.utils.web3 import Web3Provider 
from src.utils.network import client, parse_json

# constants
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
FROM_BLOCK = 13900000  # Dec-29-2021
BATCH_SIZE = 500000
API_ENDPOINT = "https://api.etherscan.io/api"

pd.set_option("expand_frame_repr", False)

In [2]:
"""get current block, load vaults from the yearn instance and initialize subgraph instance"""
load_dotenv()
w3 = Web3Provider(Network.Mainnet)
current_block = w3.provider.eth.get_block("latest")["number"]
print(f"current block number: {current_block}")

yearn = Yearn(Network.Mainnet)
vaults = yearn.vaults
print(f"loaded metadata for {len(vaults)} vaults (v2)")

subgraph = Subgraph(Network.Mainnet)

current block number: 14968136
loaded metadata for 101 vaults (v2)


In [3]:
"""get vaults, accounts and overall accounts' stats for withdrawal"""
withdrawals_stats = {"vaults": {}, "overall_account_transfers": {} }
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue

    transfers = subgraph.transfers("withdrawals", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        "count": transfers.count,
        "shares": transfers.shares,
        "shares_usdc": float(Decimal(transfers.shares) * share_price),
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        _inner_dict = {
            "count": transfer.count,
            "shares": transfer.shares,
            "shares_usdc": float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = _inner_dict

        # print top 10 accounts' stats for current vault
        if index < 10:
            print(f"{index}: {account} {_inner_dict}")

        # add overall_account_transfers to withdrawals stats
        overall_account_transfers = withdrawals_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _inner_dict[k] for k in _inner_dict}

    # add vault's stats to withdrawals stats
    withdrawals_stats["vaults"][vault.address] = {**_dict, "account_transfers": account_transfers}

vault: 0xd88dBBA3f9c4391Ee46f5FF548f289054db6E51C {'count': 54, 'shares': 6920168.562908986, 'shares_usdc': 7111843.391764439}
0: 0xfd446abde36c9dc02bb872303965484330eda047 {'count': 2, 'shares': 1986533.3281177355, 'shares_usdc': 2041556.3282399406}
1: 0x724f321c4efed5e3c7cca40168610c258c82d02f {'count': 2, 'shares': 1391595.5962067335, 'shares_usdc': 1430140.0110304677}
2: 0x2b9d8f558e02753ed7d4d97dedfad662c02af3cb {'count': 1, 'shares': 398658.769852213, 'shares_usdc': 409700.8204595796}
3: 0x0bb31c6278d58cff41b7e8ed3b20f76424fd69ad {'count': 2, 'shares': 397349.8473674423, 'shares_usdc': 408355.6434398257}
4: 0x357dfdc34f93388059d2eb09996d80f233037cba {'count': 2, 'shares': 397236.46471844433, 'shares_usdc': 408239.1203182158}
5: 0x21515f765b58bb110f68135f4a49ccd2d8a63bba {'count': 3, 'shares': 351357.0171948468, 'shares_usdc': 361088.90385710966}
6: 0x683231e17f6db6b4208206164c08e19826bc068a {'count': 1, 'shares': 304133.3161945152, 'shares_usdc': 312557.2007864709}
7: 0x0fae1e394

In [4]:
# withdrawals stats for all vaults and overall_account_transfers
for key in withdrawals_stats.keys():
    vault_stats = []
    for subkey, subval in withdrawals_stats[key].items():
        # only include address, count, shares and shares_usdc
        new_val = dict((k, v) for k, v in subval.items() if k in _dict.keys())
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top withdrawal {key}")
    print(df.head(10))
    print()

top withdrawal vaults
                                            count        shares  approx. USDC value
address                                                                            
0xdA816459F1AB5631232FE5e97a05BBBb94970c95   1700  5.452496e+08        5.454639e+08
0x27b7b1ad7288079A66d12350c828D3C00A6F07d7    346  3.214163e+08        3.344876e+08
0xa258C4606Ca8206D8aA700cE2143D7db854D168c   1809  2.919875e+05        3.244809e+08
0xdCD90C7f6324cfa40d7169ef80b12031770B4325   1289  1.977915e+05        2.291718e+08
0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE    892  2.185959e+08        2.186834e+08
0xB4AdA607B9d6b2c9Ee07A275e9616B84AC560139    166  1.462479e+08        1.474939e+08
0x5f18C75AbDAe578b483E5F43f12a39cF75b973a9    958  1.147439e+08        1.147783e+08
0x7Da96a3891Add058AdA2E826306D812C638D87a7    568  8.912520e+07        8.892930e+07
0x3B96d491f067912D18563d56858Ba7d6EC67a6fa    481  8.140770e+07        8.652515e+07
0x2DfB14E32e2F8156ec15a2c21c3A6c053af52Be8     99  7.8

In [5]:
"""get vaults, accounts and overall accounts' stats for deposits"""
deposits_stats = { "vaults": {}, "overall_account_transfers": {} }
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue
    
    transfers = subgraph.transfers("deposits", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        "count": transfers.count,
        "shares": transfers.shares,
        "shares_usdc": float(Decimal(transfers.shares) * share_price)
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        _inner_dict = {
            "count": transfer.count,
            "shares": transfer.shares,
            "shares_usdc": float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = _inner_dict

        # print top 10 accounts' stats for current vault
        if index < 10:
            print(f"{index}: {account} {_inner_dict}")

        # add overall_account_transfers to deposits stats
        overall_account_transfers = deposits_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _inner_dict[k] for k in _inner_dict}

    # add vault's stats to deposits stats
    deposits_stats["vaults"][vault.address] = {**_dict, "account_transfers": account_transfers}

vault: 0xd88dBBA3f9c4391Ee46f5FF548f289054db6E51C {'count': 87, 'shares': 19159197.744167384, 'shares_usdc': 19689869.203285333}
0: 0xa558d4aef61aacdee8ef21c11b3164cd11b273af {'count': 2, 'shares': 9837986.801675472, 'shares_usdc': 10110479.36010828}
1: 0xfd446abde36c9dc02bb872303965484330eda047 {'count': 2, 'shares': 1986533.3281177355, 'shares_usdc': 2041556.3282399406}
2: 0x724f321c4efed5e3c7cca40168610c258c82d02f {'count': 3, 'shares': 1391595.5962067335, 'shares_usdc': 1430140.0110304677}
3: 0x7d2ab9ca511ebd6f03971fb417d3492aa82513f0 {'count': 1, 'shares': 1345053.3670148808, 'shares_usdc': 1382308.6551744589}
4: 0x8e52522e6a77578904ddd7f528a22521dc4154f5 {'count': 31, 'shares': 1211068.017032276, 'shares_usdc': 1244612.1789680358}
5: 0x2b9d8f558e02753ed7d4d97dedfad662c02af3cb {'count': 1, 'shares': 398658.769852213, 'shares_usdc': 409700.8204595796}
6: 0x0bb31c6278d58cff41b7e8ed3b20f76424fd69ad {'count': 2, 'shares': 397349.8473674423, 'shares_usdc': 408355.6434398257}
7: 0x21515

In [6]:
# deposits stats for all vaults and overall_account_transfers
for key in deposits_stats.keys():
    vault_stats = []
    for subkey, subval in deposits_stats[key].items():
        # only include address, count, shares and shares_usdc
        new_val = dict((k, v) for k, v in subval.items() if k in _dict.keys())
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top deposit {key}")
    print(df.head(10))
    print()

top deposit vaults
                                            count        shares  approx. USDC value
address                                                                            
0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE   2765  4.435866e+08        4.431874e+08
0xdA816459F1AB5631232FE5e97a05BBBb94970c95   1436  3.041775e+08        3.042970e+08
0xa258C4606Ca8206D8aA700cE2143D7db854D168c   3960  1.842951e+05        2.048041e+08
0x27b7b1ad7288079A66d12350c828D3C00A6F07d7    188  1.731556e+08        1.801974e+08
0xdCD90C7f6324cfa40d7169ef80b12031770B4325   1331  1.290764e+05        1.495548e+08
0xB4AdA607B9d6b2c9Ee07A275e9616B84AC560139    110  8.031713e+07        8.100143e+07
0x5faF6a2D186448Dfa667c51CB3D695c7A6E52d8E    252  6.266090e+04        6.991078e+07
0x7Da96a3891Add058AdA2E826306D812C638D87a7    885  6.876278e+07        6.872840e+07
0x67e019bfbd5a67207755D04467D6A70c0B75bF60    294  4.483668e+07        4.718617e+07
0x2DfB14E32e2F8156ec15a2c21c3A6c053af52Be8     82  4.6702

In [7]:
"""helper functions to calculate net transfers"""

# Calculate net transfers of these keys
keys = ["count", "shares", "shares_usdc"]

def equalize_lists(*args: list[tuple]) -> None:
    """Make all lists equal in size"""
    items = [*args]
    max_length = 0
    for item in items:
        max_length = max(max_length, len(item))
    for item in items:
        item += [(None, None)] * (max_length - len(item))

def get_net_transfer(withdrawals_items: list[tuple[str, dict]], deposits_items: list[tuple[str, dict]]) -> dict[str, dict[str, float]]:
    """Get net transfer of vaults and overall account transfers"""
    net_transfer = {}

    for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_items, deposits_items):
        if withdraw_address is not None:
            if net_transfer.get(withdraw_address) is None:
                net_transfer[withdraw_address] = {}
            _dict = {}
            for k in keys:
                if k == "count":
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) + withdraw_stats[k]
                else:
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) - withdraw_stats[k]
            net_transfer[withdraw_address] = _dict
        if deposit_address is not None:
            if net_transfer.get(deposit_address) is None:
                net_transfer[deposit_address] = {}
            net_transfer[deposit_address] = {k: net_transfer[deposit_address].get(k, 0) + deposit_stats[k] for k in keys}

    return net_transfer

In [8]:
"""net transfer of each vault's transfer count, shares and shares usdc"""
withdrawals_vaults = list(withdrawals_stats["vaults"].items())
deposits_vaults = list(deposits_stats["vaults"].items())
equalize_lists(withdrawals_vaults, deposits_vaults)
net_vaults = get_net_transfer(withdrawals_vaults, deposits_vaults)

In [9]:
"""net transfer of each account's transfer count, shares and shares usdc scoped by all vaults"""
withdrawals_account_transfers = list(withdrawals_stats["overall_account_transfers"].items())
deposits_account_transfers = list(deposits_stats["overall_account_transfers"].items())
equalize_lists(withdrawals_account_transfers, deposits_account_transfers)
net_overall_account_transfers = get_net_transfer(withdrawals_account_transfers, deposits_account_transfers)

In [10]:
"""net transfer of each vault's account's transfer count, shares and shares usdc scoped by each vault"""
net_transfer = {}
for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_vaults, deposits_vaults):
    if withdraw_address is not None:
        if net_transfer.get(withdraw_address) is None:
            net_transfer[withdraw_address] = {}

        for (account, account_transfer) in list(withdraw_stats["account_transfers"].items()):
            if net_transfer[withdraw_address].get(account) is None:
                net_transfer[withdraw_address][account] = {}
            _dict = {}
            for k in keys:
                if k == "count":
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) + account_transfer[k]
                else:
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) - account_transfer[k]
            net_transfer[withdraw_address][account] = _dict
        
    if deposit_address is not None:
        if net_transfer.get(deposit_address) is None:
            net_transfer[deposit_address] = {}

        for (account, account_transfer) in list(deposit_stats["account_transfers"].items()):
            if net_transfer[deposit_address].get(account) is None:
                net_transfer[deposit_address][account] = {}
            account_val = {k: net_transfer[deposit_address][account].get(k, 0) + account_transfer[k] for k in keys}
            net_transfer[deposit_address][account] = account_val